In [94]:
#Required libraries
import numpy as np
import cv2
import os
import random

In [95]:
# Canvas settings
canvas_size = (500, 600, 3)  # Height, Width, Channels
sky_color = (255, 255, 204)  # Light yellowish sky

# Helper functions
def draw_sun(img):
    # Draw the sun
    cv2.circle(img, (50, 50), 40, (0, 255, 255), -1)
    
    # Draw sun rays
    for angle in range(0, 360, 15):  # Draw rays at different angles
        x1 = int(50 + 50 * np.cos(np.radians(angle)))
        y1 = int(50 + 50 * np.sin(np.radians(angle)))
        x2 = int(50 + 70 * np.cos(np.radians(angle)))
        y2 = int(50 + 70 * np.sin(np.radians(angle)))
        cv2.line(img, (x1, y1), (x2, y2), (0, 255, 255), 2)

def draw_cloud(img, center, size):
    x, y = center
    for offset in [(-size, 0), (0, -size // 2), (size, 0)]:
        cv2.circle(img, (x + offset[0], y + offset[1]), size, (255, 255, 255), -1)

def draw_tree(img, trunk_bottom, trunk_width, leaves_radius):
    x, y = trunk_bottom
    cv2.rectangle(img, (x - trunk_width // 2, y - 100), (x + trunk_width // 2, y), (42, 42, 165), -1)  # Trunk
    leaf_positions = [
        (x, y - 100),
        (x - leaves_radius, y - 120),
        (x + leaves_radius, y - 120),
        (x, y - 140)
    ]
    for pos in leaf_positions:
        cv2.circle(img, pos, leaves_radius, (34, 139, 34), -1)  # Leaves

def draw_crow(img, position, carrying=False, pebble=None):
    x, y = position
    # Draw the body
    cv2.circle(img, (x, y), 10, (0, 0, 0), -1)  # Body
    # Draw the beak
    cv2.line(img, (x, y), (x + 15, y - 10), (0, 0, 0), 2)  # Beak
    # Draw the eyes
    cv2.circle(img, (x - 3, y - 5), 2, (255, 255, 255), -1)  # Left eye
    cv2.circle(img, (x + 3, y - 5), 2, (255, 255, 255), -1)  # Right eye
    
    # Decreased size wings
    cv2.line(img, (x - 10, y), (x - 15, y - 15), (0, 0, 0), 2)  # Left wing (shorter)
    cv2.line(img, (x + 10, y), (x + 15, y - 15), (0, 0, 0), 2)  # Right wing (shorter)
    
    if carrying and pebble:
        cv2.circle(img, pebble, 5, (128, 128, 128), -1)

def draw_pot(img, water_level, pebbles=[]):
    # Draw the pot
    cv2.ellipse(img, (300, 380), (35, 45), 0, 0, 180, (128, 128, 128), -1)
    cv2.rectangle(img, (270, 380), (330, 450), (128, 128, 128), -1)
    cv2.rectangle(img, (280, water_level), (320, 450), (255, 0, 0), -1)  # Water

    # Draw pebbles in the water area
    for pebble in pebbles:
        if water_level < pebble[1] < 450:  # Ensure the pebble is inside the water
            cv2.circle(img, pebble, 5, (128, 128, 128), -1)

def draw_background():
    img = np.zeros(canvas_size, dtype=np.uint8)
    img[:] = sky_color
    cv2.rectangle(img, (0, 300), (600, 400), (173, 216, 230), -1)  # Ground
    cv2.rectangle(img, (0, 400), (600, 500), (34, 139, 34), -1)  # Grass
    draw_sun(img)
    
    # Add multiple clouds
    draw_cloud(img, (150, 80), 20)
    draw_cloud(img, (400, 50), 30)
    draw_cloud(img, (random.randint(300, 500), random.randint(50, 150)), random.randint(15, 25))  # New cloud

    # Add natural-looking trees
    draw_tree(img, (100, 350), 20, 40)
    draw_tree(img, (200, 370), 15, 35)
    draw_tree(img, (450, 380), 25, 50)

    return img

def add_text(img, text):
    cv2.putText(img, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2, cv2.LINE_AA)

# Animation settings
output_folder = "frames"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

pebble_positions = [(250, 420), (230, 430), (210, 440), (190, 450)]  # Pebbles picked up by the crow
water_levels = [430, 420, 410, 400]  # Decrease in water level

frames = []
def save_frame(img, frame_id):
    frame_path = os.path.join(output_folder, f"frame_{frame_id}.jpg")
    cv2.imwrite(frame_path, img)
    frames.append(frame_path)

# Generate frames
frame_id = 0

In [96]:
#Generating Frames as per story

# Step 1: Crow flying in the sky
img = draw_background()
draw_crow(img, (300, 100))
add_text(img, "The crow is flying in the sky and gets thirsty.")
save_frame(img, frame_id)
frame_id += 1

# Step 2: Crow sees the pot
img = draw_background()
draw_crow(img, (300, 200))
draw_pot(img, water_levels[0], [])
add_text(img, "The crow sees the pot and flies towards it.")
save_frame(img, frame_id)
frame_id += 1

# Step 3: Crow near the pot, examining
img = draw_background()
draw_crow(img, (300, 350))
draw_pot(img, water_levels[0], [])
add_text(img, "The crow finds the water level is too low.")
save_frame(img, frame_id)
frame_id += 1

# Step 4: Crow picks up pebbles and drops them
dropped_pebbles = []
for i, pebble in enumerate(pebble_positions):
    # Crow picking pebble
    img = draw_background()
    draw_crow(img, pebble, carrying=True, pebble=pebble)
    draw_pot(img, water_levels[i], dropped_pebbles)
    add_text(img, "The crow is picking up a stone.")
    save_frame(img, frame_id)
    frame_id += 1

    # Crow dropping pebble inside the pot
    # Place the pebble slightly below the water level
    dropped_pebbles.append((300, water_levels[i] + 10))  # Dropping the pebble below the water level
    img = draw_background()
    draw_crow(img, (300, 350), carrying=True, pebble=pebble)
    draw_pot(img, water_levels[i], dropped_pebbles)
    add_text(img, "The crow drops the stone into the pot.")
    save_frame(img, frame_id)
    frame_id += 1

# Step 5: Water level rises, crow drinks
img = draw_background()
draw_crow(img, (300, 350))
draw_pot(img, water_levels[-1], dropped_pebbles)
add_text(img, "The crow drinks the water and thanks nature.")
save_frame(img, frame_id)
frame_id += 1


# Step 6: Add credits
img = np.zeros(canvas_size, dtype=np.uint8)  # Black background for credits
cv2.putText(img, "Created by: Nikhil Karnati", (15, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 150, 255), 2, cv2.LINE_AA)
cv2.putText(img, "Under the supervision of: Saxon sir", (15, 250), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
cv2.putText(img, "Mentor: Lakshmiteja mam", (15, 300), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 105, 80), 2, cv2.LINE_AA)
save_frame(img, frame_id)
frame_id += 1

In [97]:
#To display the frames as gif

# Display the animation with all frames
c = 0
while True:
    img_path = frames[c]
    img = cv2.imread(img_path)
    cv2.imshow("Crow Story Animation", img)

    if cv2.waitKey(2500) & 0xFF == ord("q"):  # Quit animation on 'q'
        break

    c = (c + 1) % len(frames)

cv2.destroyAllWindows()